# Libraries

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import utils
import pandas as pd
import scipy.signal as signal
import random 
import pywt
from sklearn.cross_decomposition import CCA

# Initial Set Up

Here I am just making sure everything is running properly and the libraries are correctly imported.

In [21]:
low_freq = 5
high_freq = 40
sampling_rate = 250

b, a = signal.butter(4, [low_freq, high_freq], fs=sampling_rate, btype='band')

In [22]:
sampling_rate = 250
data_frequencies, electrodes_names = utils.load_ssvep_additional_info("../../TFM-GitHub/Dataset BETA/S1")
occipital_electrodes, occipital_electrodes_indexes, occipital_electrodes_names = utils.select_occipital_electrodes(electrodes_names)

samples_without_stimulus = int(0.5 * sampling_rate)

In [23]:
window_length = int(750/3) #We chose 750 because it is the shortest trial duration
window_step = 10 #int(750/24)

print("Window length = ",window_length)
print("Window step = ", window_step)
print("Number of steps = ", len(np.arange(0, 750, step=window_step)), " and ", len(np.arange(0, 1000, step=window_step)))

Window length =  250
Window step =  10
Number of steps =  75  and  100


In [24]:
# Subject with the highest ITR in the BETA article is: Subject 67
# Subject with the lowest ITR in the BETA article is: Subject 11

In [25]:
# Creating an array with the ITRs for each subject
subjects_ITR = [139.63, 148.49, 143.02, 98.37, 134.44, 85.97, 120.77, 87.48, 132.28, 77.22,
                 32.30, 136.97, 125.03, 103.00, 131.44, 94.84, 54.38, 182.66, 116.83, 73.09, 
                 136.39, 122.74, 192.63, 109.57, 105.62, 84.67, 120.75, 116.35, 109.42, 117.79, 
                 64.19, 72.39, 61.61, 128.43, 120.96, 131.44, 148.84, 72.80, 100.83, 87.37, 
                 38.64, 132.80, 82.46, 46.05, 94.24, 84.49, 58.15, 144.17, 180.56, 75.67, 
                 99.62, 136.78, 78.14, 86.02, 33.68, 138.09, 132.46, 113.79, 54.76, 111.07, 
                 48.94, 89.35, 163.57, 81.16, 56.19, 150.49, 207.16, 117.60, 112.14, 123.50]

In [26]:
print(len(subjects_ITR))
print("Maximum ITR ", max(subjects_ITR), "belongs to subject ", np.argmax(subjects_ITR) + 1)
print("Minimum ITR ", min(subjects_ITR), "belongs to subject ", np.argmin(subjects_ITR) + 1)

70
Maximum ITR  207.16 belongs to subject  67
Minimum ITR  32.3 belongs to subject  11


# Sliding Windows - normal FFT

In [91]:
window_length = int(750/3) #We chose 750 because it is the shortest trial duration
window_step = 10 #int(750/24)

print("Window length = ",window_length)
print("Window step = ", window_step)
print("Number of steps = ", len(np.arange(0, 750, step=window_step)), " and ", len(np.arange(0, 1000, step=window_step)))

Window length =  250
Window step =  10
Number of steps =  75  and  100


In [92]:
electrode_idx = 61 #Electrode Oz

sliding_detections_psda = np.ones((70, 4, 40)) * -1

for subject in range(1,71):
    
    eeg_data, eeg_data_stimulus_only, trial_length = utils.load_ssvep_data("../../TFM-GitHub/Dataset BETA/",subject)

    max_length = len(eeg_data[electrode_idx, :, 0, 0])

    for trial_idx in range(4):

        for freq_idx in range(40):

            detected = False

            ini = int(250*0.5)
            end = ini + window_length
            step = 0

            filtered_eeg = signal.lfilter(b, a, eeg_data[electrode_idx, : , trial_idx, freq_idx])

            fft_result = np.fft.fft(filtered_eeg)
            freq = np.fft.fftfreq(len(filtered_eeg), d=1/sampling_rate)
            power_spectra = np.abs(fft_result) ** 2

            result = np.abs(freq[np.argmax(power_spectra)])

            while(detected == False and end < max_length):

                filtered_eeg = signal.lfilter(b, a, eeg_data[electrode_idx, ini:end, trial_idx, freq_idx])

                fft_result = np.fft.fft(filtered_eeg)
                freq = np.fft.fftfreq(len(filtered_eeg), d=1/sampling_rate)
                power_spectra = np.abs(fft_result) ** 2

                result = np.abs(freq[np.argmax(power_spectra)])

                if result % data_frequencies[freq_idx] == 0:
                    detected = True
                    sliding_detections_psda[subject-1, trial_idx, freq_idx] = ini

                ini += window_step
                end += window_step
                step += 1
            



In [93]:
#Store the data into a file

utils.store_data("sliding_psda.h5", sliding_detections_psda)

Data stored succesfully in sliding_psda.h5


In [94]:
def not_minus_one(val):
    if val != -1:
        return 'color: green'
    else:
        return 'color: red'
    
df = pd.DataFrame(sliding_detections_psda[54, :, :], index=["T1", "T2", "T3", "T4"], columns = data_frequencies)
styled_df = df.style.applymap(not_minus_one)
formatted_df = styled_df.format(formatter="{:.0f}")
display(formatted_df)


/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/4062784934.py:8: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)


,8.600000,8.800000,9.000000,9.200000,9.400000,9.600000,9.800000,10.000000,10.200000,10.400000,10.600000,10.800000,11.000000,11.200000,11.400000,11.600000,11.800000,12.000000,12.200000,12.400000,12.600000,12.800000,13.000000,13.200000,13.400000,13.600000,13.800000,14.000000,14.200000,14.400000,14.600000,14.800000,15.000000,15.200000,15.400000,15.600000,15.800000,8.000000,8.200000,8.400000
T1,-1,-1,125,-1,-1,-1,-1,205,-1,-1,-1,-1,125,-1,-1,-1,-1,175,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,125,-1,-1,-1,-1,505,-1,-1
T2,-1,-1,295,-1,-1,-1,-1,125,-1,-1,-1,-1,135,-1,-1,-1,-1,225,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,165,-1,-1,-1,-1,275,-1,-1
T3,-1,-1,215,-1,-1,-1,-1,185,-1,-1,-1,-1,125,-1,-1,-1,-1,365,-1,-1,-1,-1,145,-1,-1,-1,-1,435,-1,-1,-1,-1,685,-1,-1,-1,-1,145,-1,-1
T4,-1,-1,155,-1,-1,-1,-1,295,-1,-1,-1,-1,125,-1,-1,-1,-1,345,-1,-1,-1,-1,-1,-1,-1,-1,-1,615,-1,-1,-1,-1,175,-1,-1,-1,-1,215,-1,-1


In [95]:
table_psda = np.zeros((70, 3))

for subject in range(70):
    time = 0
    accuracy = 0
    for trial in range(4):
        
        for f in range(40):
            if sliding_detections_psda[subject, trial, f] != -1:
                time += sliding_detections_psda[subject, trial, f]/sampling_rate
                accuracy += 1
            #elif subject < 15:
            #    time += 75
            #else:
            #    time += 100
    table_psda[subject, 0] = time/accuracy
    table_psda[subject, 1] = accuracy*100/160
    table_psda[subject, 2] = subjects_ITR[subject]
    


def color_condition(val, threshold, color_1, color_2):
    if val >= threshold:
        return 'color: '+color_1
    else:
        return 'color: '+color_2
    
df = pd.DataFrame(table_psda, index=range(1, 71), columns = ["Average time", "Accuracy %", "ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/4005040118.py:29: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/4005040118.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/4005040118.py:32: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/4005040118.py:33: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])


,Average time,Accuracy %,ITR
1,0.59,20.00,139.63
2,0.94,17.50,148.49
3,0.58,20.00,143.02
4,0.92,13.75,98.37
5,0.99,13.12,134.44
6,0.82,13.75,85.97
7,0.95,15.00,120.77
8,0.84,17.50,87.48
9,0.56,20.00,132.28
10,0.61,18.12,77.22


In [96]:
T = 0.55
M = 40

my_ITR_psda = []
for subject in range(70):
    P = table_psda[subject, 1]/100
    t = table_psda[subject, 0]
    if P == 1: P = 0.999999999
    value = 60 * (np.log(M) + P * np.log(P) + (1 - P) * np.log((1 - P) / (M - 1))) / (T + t)
    my_ITR_psda.append(value)



final_table_m_psda = np.hstack((table_psda, np.array(my_ITR_psda)[:, np.newaxis]))

df = pd.DataFrame(final_table_m_psda, index=range(1, 71), columns = ["Average time", "Accuracy %", "Article ITR", "My ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 1, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['My ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/714533987.py:17: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/714533987.py:19: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 1, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/714533987.py:20: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/714533987.py:21: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
/

,Average time,Accuracy %,Article ITR,My ITR
1,0.59,20.00,139.63,13.57
2,0.94,17.50,148.49,8.15
3,0.58,20.00,143.02,13.72
4,0.92,13.75,98.37,5.24
5,0.99,13.12,134.44,4.58
6,0.82,13.75,85.97,5.64
7,0.95,15.00,120.77,6.09
8,0.84,17.50,87.48,8.77
9,0.56,20.00,132.28,13.94
10,0.61,18.12,77.22,11.21


# Sliding Windows - modified FFT

In [97]:
window_length = int(750/3) #We chose 750 because it is the shortest trial duration
window_step = 10 #int(750/24)

print("Window length = ",window_length)
print("Window step = ", window_step)
print("Number of steps = ", len(np.arange(0, 750, step=window_step)), " and ", len(np.arange(0, 1000, step=window_step)))

Window length =  250
Window step =  10
Number of steps =  75  and  100


In [98]:
electrode_idx = 61 #Electrode Oz

sliding_detections_m_psda = np.ones((70, 4, 40)) * -1

for subject in range(1,71):
    
    eeg_data, eeg_data_stimulus_only, trial_length = utils.load_ssvep_data("../../TFM-GitHub/Dataset BETA/",subject)

    max_length = len(eeg_data[electrode_idx, :, 0, 0])

    for trial_idx in range(4):

        for freq_idx in range(40):

            detected = False

            ini = int(250*0.5)
            end = ini + window_length

            filtered_eeg = signal.lfilter(b, a, eeg_data[electrode_idx, : , trial_idx, freq_idx])

            fft_result = np.fft.fft(filtered_eeg)
            freq = np.fft.fftfreq(len(filtered_eeg), d=1/sampling_rate)
            power_spectra = np.abs(fft_result) ** 2

            result = np.abs(freq[np.argmax(power_spectra)])

            step = 0

            while(detected == False and end < max_length):

                filtered_eeg = signal.lfilter(b, a, eeg_data[electrode_idx, ini:end, trial_idx, freq_idx])

                fft_result = np.fft.fft(filtered_eeg)
                freq = np.fft.fftfreq(len(filtered_eeg), d=1/sampling_rate)
                power_spectra = np.abs(fft_result) ** 2

                peak = np.abs(freq[np.argmax(power_spectra)])

                detected_frequency = 0
                min_module = 100000
                for i in range(40):
                    if peak % data_frequencies[i] < min_module:
                        detected_frequency = data_frequencies[i]
                        min_module = peak % detected_frequency

                if detected_frequency == data_frequencies[freq_idx]:
                    detected = True
                    sliding_detections_m_psda[subject-1, trial_idx, freq_idx] = ini

                ini += window_step
                end += window_step
                step += 1



In [99]:
#Store the data into a file

utils.store_data("sliding_m_psda.h5", sliding_detections_m_psda)

Data stored succesfully in sliding_m_psda.h5


In [100]:

def not_minus_one(val):
    if val != -1:
        return 'color: green'
    else:
        return 'color: red'
    
df = pd.DataFrame(sliding_detections_m_psda[54, :, :], index=["T1", "T2", "T3", "T4"], columns = data_frequencies)
styled_df = df.style.applymap(not_minus_one)
formatted_df = styled_df.format(formatter="{:.0f}")
display(formatted_df)


/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/3480898211.py:8: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)


,8.600000,8.800000,9.000000,9.200000,9.400000,9.600000,9.800000,10.000000,10.200000,10.400000,10.600000,10.800000,11.000000,11.200000,11.400000,11.600000,11.800000,12.000000,12.200000,12.400000,12.600000,12.800000,13.000000,13.200000,13.400000,13.600000,13.800000,14.000000,14.200000,14.400000,14.600000,14.800000,15.000000,15.200000,15.400000,15.600000,15.800000,8.000000,8.200000,8.400000
T1,125,-1,125,-1,-1,-1,-1,205,-1,-1,-1,-1,125,-1,595,-1,-1,175,-1,-1,495,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,545,-1,-1,505,-1,-1
T2,315,-1,295,-1,-1,-1,-1,125,-1,-1,-1,-1,135,-1,-1,-1,-1,225,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,545,-1,-1,-1,-1,275,-1,-1
T3,135,-1,215,-1,-1,-1,-1,185,-1,-1,-1,-1,125,-1,-1,-1,-1,365,-1,-1,-1,-1,145,-1,-1,-1,-1,435,-1,-1,-1,-1,685,-1,-1,-1,-1,145,-1,515
T4,185,-1,155,-1,-1,-1,-1,295,-1,-1,-1,-1,125,-1,-1,-1,-1,345,-1,-1,-1,-1,-1,-1,-1,-1,-1,615,-1,-1,-1,-1,175,-1,-1,-1,-1,215,-1,-1


In [101]:
table_m_psda = np.zeros((70, 3))

for subject in range(70):
    time = 0
    accuracy = 0
    for trial in range(4):
        
        for f in range(40):
            if sliding_detections_m_psda[subject, trial, f] != -1:
                time += sliding_detections_psda[subject, trial, f]/sampling_rate
                accuracy += 1
            #elif subject < 15:
            #    time += 75
            #else:
            #    time += 100
    table_m_psda[subject, 0] = time/accuracy
    table_m_psda[subject, 1] = accuracy*100/160
    table_m_psda[subject, 2] = subjects_ITR[subject]
    


def color_condition(val, threshold, color_1, color_2):
    if val >= threshold:
        return 'color: '+color_1
    else:
        return 'color: '+color_2
    
df = pd.DataFrame(table_m_psda, index=range(1, 71), columns = ["Average time", "Accuracy %", "ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/1162712481.py:29: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/1162712481.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/1162712481.py:32: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/1162712481.py:33: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])


,Average time,Accuracy %,ITR
1,0.37,31.88,139.63
2,0.61,26.88,148.49
3,0.37,30.62,143.02
4,0.78,16.25,98.37
5,0.80,16.25,134.44
6,0.42,26.88,85.97
7,0.76,18.75,120.77
8,0.63,23.12,87.48
9,0.45,25.00,132.28
10,0.45,24.38,77.22


In [102]:
T = 0.55
M = 40

my_ITR_m_psda = []
for subject in range(70):
    P = table_m_psda[subject, 1]/100
    t = table_m_psda[subject, 0]
    if P == 1: P = 0.999999999
    value = 60 * (np.log(M) + P * np.log(P) + (1 - P) * np.log((1 - P) / (M - 1))) / (T + t)
    my_ITR_m_psda.append(value)



final_table_m_psda = np.hstack((table_m_psda, np.array(my_ITR_m_psda)[:, np.newaxis]))

df = pd.DataFrame(final_table_m_psda, index=range(1, 71), columns = ["Average time", "Accuracy %", "Article ITR", "My ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 1, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['My ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/731191277.py:17: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/731191277.py:19: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 1, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/731191277.py:20: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_34169/731191277.py:21: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
/

,Average time,Accuracy %,Article ITR,My ITR
1,0.37,31.88,139.63,37.07
2,0.61,26.88,148.49,22.08
3,0.37,30.62,143.02,34.46
4,0.78,16.25,98.37,7.97
5,0.80,16.25,134.44,7.87
6,0.42,26.88,85.97,26.56
7,0.76,18.75,120.77,10.54
8,0.63,23.12,87.48,16.83
9,0.45,25.00,132.28,22.82
10,0.45,24.38,77.22,21.78


# Sliding Windows - CCA 

In [153]:
def find_correlation(data, reference_signals, data_frequencies):
    cca = CCA(n_components=1)
    max_correlation = -np.inf
    most_correlated_frequency = None
    
    for i, ref_signal_set in enumerate(reference_signals):
        X = np.array(ref_signal_set).reshape(-1, 12)  # Reshape reference signals
        Y = data.T  # Transpose EEG data if necessary
        
        cca.fit(X, Y)
        x_scores, y_scores = cca.transform(X, Y)
        correlation = np.corrcoef(x_scores[:, 0], y_scores[:, 0])[0, 1]
        
        if correlation > max_correlation:
            max_correlation = correlation
            most_correlated_frequency = i
    
    return data_frequencies[most_correlated_frequency], max_correlation

In [154]:
duration = window_length/sampling_rate
ref_signals = []
for i in range(40):
    aux = np.array(utils.generate_ref_signal(data_frequencies[i], sampling_rate, duration, 6, 0))
    ref_signals.append(aux)
ref_signals = np.array(ref_signals)

In [155]:
sliding_detections_cca = []

electrode_idx = 61 #Electrode Oz

sliding_detections_cca = np.ones((70, 4, 40)) * -1

for subject in range(1,71):
    
    eeg_data, eeg_data_stimulus_only, trial_length = utils.load_ssvep_data("../../TFM-GitHub/Dataset BETA/",subject)

    max_length = len(eeg_data[electrode_idx, :, 0, 0])

    for trial_idx in range(4):

        for freq_idx in range(40):

            detected = False

            ini = int(250 * 0.5)
            end = ini + window_length

            filtered_eeg = signal.lfilter(b, a, eeg_data[electrode_idx, : , trial_idx, freq_idx])

            step = 0

            while(detected == False and end < max_length):

                prediction, _ = find_correlation(filtered_eeg[ini:end], ref_signals, data_frequencies)
                if prediction == data_frequencies[freq_idx]:
                    detected = True
                    sliding_detections_cca[subject-1, trial_idx, freq_idx] = ini
                
                ini += window_step
                end += window_step
                step += 1



In [156]:
#Store the data into a file

utils.store_data("sliding_cca.h5", sliding_detections_cca)

Data stored succesfully in sliding_cca.h5


In [28]:

def not_minus_one(val):
    if val != -1:
        return 'color: green'
    else:
        return 'color: red'
    
df = pd.DataFrame(sliding_detections_cca[17, :, :], index=["T1", "T2", "T3", "T4"], columns = data_frequencies)
styled_df = df.style.applymap(not_minus_one)
formatted_df = styled_df.format(formatter="{:.0f}")
display(formatted_df)


/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3459549708.py:8: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)


,8.600000,8.800000,9.000000,9.200000,9.400000,9.600000,9.800000,10.000000,10.200000,10.400000,10.600000,10.800000,11.000000,11.200000,11.400000,11.600000,11.800000,12.000000,12.200000,12.400000,12.600000,12.800000,13.000000,13.200000,13.400000,13.600000,13.800000,14.000000,14.200000,14.400000,14.600000,14.800000,15.000000,15.200000,15.400000,15.600000,15.800000,8.000000,8.200000,8.400000
T1,255,125,125,175,125,125,125,175,165,125,275,185,185,125,195,145,155,145,125,285,155,135,125,215,125,135,175,145,235,185,145,165,185,135,125,145,-1,125,135,125
T2,155,255,135,135,125,225,185,165,125,165,125,285,165,155,145,135,135,155,175,125,125,275,145,135,125,175,145,135,145,125,175,125,125,165,205,125,205,135,145,125
T3,135,125,125,125,125,125,125,135,165,145,125,665,125,145,135,225,135,125,125,125,125,175,155,175,125,175,125,165,125,145,125,185,145,135,265,125,385,195,125,125
T4,145,215,125,125,165,255,125,125,125,155,145,145,185,135,125,175,125,145,135,165,185,165,155,275,175,155,145,255,265,125,155,205,385,255,175,205,195,135,155,145


In [36]:
table_cca = np.zeros((70, 3))

for subject in range(70):
    time = 0
    accuracy = 0
    for trial in range(4):
        
        for f in range(40):
            if sliding_detections_cca[subject, trial, f] != -1:
                time += sliding_detections_cca[subject, trial, f]/sampling_rate
                accuracy += 1
            #elif subject < 15:
            #    time += 75
            #else:
            #    time += 100
            
    table_cca[subject, 0] = time/accuracy
    table_cca[subject, 1] = accuracy*100/160
    table_cca[subject, 2] = subjects_ITR[subject]
    


def color_condition(val, threshold, color_1, color_2):
    if val >= threshold:
        return 'color: '+color_1
    else:
        return 'color: '+color_2
    
df = pd.DataFrame(table_cca, index=range(1, 71), columns = ["Average time", "Accuracy %", "ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3530166524.py:30: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3530166524.py:32: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 10, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3530166524.py:33: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3530166524.py:34: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['ITR'])


,Average time,Accuracy %,ITR
1,0.64,96.88,139.63
2,0.92,66.25,148.49
3,0.64,97.50,143.02
4,1.02,75.62,98.37
5,1.11,58.75,134.44
6,0.95,63.12,85.97
7,1.14,55.62,120.77
8,0.89,78.75,87.48
9,0.68,96.88,132.28
10,0.76,97.50,77.22


In [37]:
aux = table_cca[:, 1]
below_sixty = []
full_success = []
x = []
y = []
for i in range(70):
    if aux[i] < 60: below_sixty.append(i+1)
    if aux[i] == 100: 
        full_success.append(i+1)
        x.append(i)
    if aux[i] == 100 and i > 15:
        y.append(i)

print("Subjects with accuracy lower than 60% ", below_sixty)
print("Subjects with 100% accuracy", full_success)

Subjects with accuracy lower than 60%  [5, 7, 11, 17, 32, 33, 41, 53, 55, 64, 69]
Subjects with 100% accuracy [19, 23, 57]


# ITR

In [38]:
T = 0.55
M = 40

my_ITR_CCA = []
for subject in range(70):
    P = table_cca[subject, 1]/100
    t = table_cca[subject, 0] #- 0.5
    if P == 1: P = 0.999999999
    value = 60 * (np.log(M) + P * np.log(P) + (1 - P) * np.log((1 - P) / (M - 1))) / (T + t)
    my_ITR_CCA.append(value)

In [39]:
final_table_cca = np.hstack((table_cca, np.array(my_ITR_CCA)[:, np.newaxis]))

df = pd.DataFrame(final_table_cca, index=range(1, 71), columns = ["Average time", "Accuracy %", "Article ITR", "My ITR"])
styled_df = df.style.applymap(not_minus_one)

styled_df = df.style.applymap(lambda x: color_condition(x, 2, 'red', 'green'), subset=['Average time'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['My ITR'])

formatted_df = styled_df.format(formatter="{:.2f}")
display(formatted_df)



/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3369403881.py:4: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(not_minus_one)
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3369403881.py:6: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(lambda x: color_condition(x, 2, 'red', 'green'), subset=['Average time'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3369403881.py:7: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 20, 'green', 'red'), subset=['Accuracy %'])
/var/folders/v_/yyjjwqx93jj6f359gjx0mwsh0000gn/T/ipykernel_73952/3369403881.py:8: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = styled_df.applymap(lambda x: color_condition(x, 100, 'green', 'red'), subset=['Article ITR'])
/

,Average time,Accuracy %,Article ITR,My ITR
1,0.64,96.88,139.63,173.79
2,0.92,66.25,148.49,73.81
3,0.64,97.50,143.02,175.37
4,1.02,75.62,98.37,85.70
5,1.11,58.75,134.44,54.37
6,0.95,63.12,85.97,66.98
7,1.14,55.62,120.77,49.00
8,0.89,78.75,87.48,100.06
9,0.68,96.88,132.28,167.70
10,0.76,97.50,77.22,158.91


In [33]:
sliding_detections_cca[17, :, 12]
print(180/250)

0.72


# Specific times

In [18]:
#Load data
sliding_detections_m_psda = utils.load_data("sliding_m_psda.h5")
sliding_detections_psda = utils.load_data("sliding_psda.h5")
sliding_detections_cca = utils.load_data("sliding_cca.h5")

Data loaded successfully from sliding_m_psda.h5
Data loaded successfully from sliding_psda.h5
Data loaded successfully from sliding_cca.h5


In [6]:
def print_trial_time(subject, freq_idx):
    for i in range(4):
        if (sliding_detections_cca[subject-1, i, freq_idx] != -1):
            print("Subject ", subject, "sample for frequency ", data_frequencies[freq_idx], "Hz is ", sliding_detections_cca[subject-1, i, freq_idx])
        else:
            print("Failed to detect the frequency ", data_frequencies[freq_idx], "Hz with CCA for subject ", subject) 

def detected_sample(subject, freq_idx, trial, data):   
    return data[subject-1, trial, freq_idx]    

## PSDA

In [7]:
samples = []

subjects = [18, 64, 11, 67]
freq_indexes = [12, 22, 0, 35]

for subject in subjects:
    for freq_idx in freq_indexes:
        for tr in range(4):
            samples.append(detected_sample(subject, freq_idx, tr, sliding_detections_psda))

print(samples)

[125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 215.0, 125.0, 125.0, 205.0, 605.0, 555.0, 415.0, 165.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 195.0, 135.0, 355.0, 355.0, 125.0, 345.0, 235.0, 355.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 125.0, 235.0, 325.0, 125.0, 135.0, 125.0, 285.0, 225.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [8]:
psda_samples = samples

## m_PSDA

In [9]:
samples = []

subjects = [18, 64, 11, 67]
freq_indexes = [12, 22, 0, 35]

for subject in subjects:
    for freq_idx in freq_indexes:
        for tr in range(4):
            samples.append(detected_sample(subject, freq_idx, tr, sliding_detections_m_psda))

print(samples)

[125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, -1.0, -1.0, 125.0, -1.0, -1.0, -1.0, -1.0, -1.0, 215.0, 125.0, 125.0, 205.0, 605.0, 555.0, 415.0, 165.0, -1.0, 135.0, -1.0, 395.0, -1.0, -1.0, -1.0, -1.0, 195.0, 135.0, 355.0, 355.0, 125.0, 345.0, 235.0, 355.0, 125.0, 125.0, 255.0, 185.0, -1.0, -1.0, -1.0, -1.0, 125.0, 235.0, 325.0, 125.0, 135.0, 125.0, 285.0, 225.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [10]:
m_psda_samples = samples

## CCA

In [11]:
samples = []

subjects = [18, 64, 11, 67]
freq_indexes = [12, 22, 0, 35]

for subject in subjects:
    for freq_idx in freq_indexes:
        for tr in range(4):
            samples.append(detected_sample(subject, freq_idx, tr, sliding_detections_cca))

print(samples)

[185.0, 165.0, 125.0, 185.0, 125.0, 145.0, 155.0, 155.0, 255.0, 155.0, 135.0, 145.0, 145.0, 125.0, 125.0, 205.0, 215.0, 125.0, 135.0, 215.0, 605.0, 585.0, 445.0, 275.0, -1.0, -1.0, 685.0, -1.0, 525.0, -1.0, -1.0, -1.0, 185.0, -1.0, -1.0, -1.0, 465.0, -1.0, -1.0, 395.0, 415.0, 495.0, 235.0, 175.0, 405.0, -1.0, -1.0, 325.0, 125.0, 205.0, 325.0, 125.0, 395.0, 185.0, 565.0, 405.0, 125.0, 125.0, 145.0, 125.0, 485.0, 545.0, 645.0, 625.0]


In [12]:
cca_samples = samples

# Shaded images

In [13]:
# Original data for two subjects

#psda_samples = [20.0, 20.0, 0.0, 0.0, 0.0, 0.0, 90.0, 0.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 210.0, 0.0, 0.0, 0.0, 610.0, 550.0, 50.0, 160.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]

#m_psda_samples = [20.0, 20.0, 0.0, 0.0, 0.0, 0.0, 90.0, 0.0, -1.0, -1.0, 0.0, 500.0, -1.0, -1.0, -1.0, -1.0, 210.0, 0.0, 0.0, 0.0, 610.0, 550.0, 50.0, 160.0, -1.0, 140.0, -1.0, 390.0, -1.0, -1.0, -1.0, -1.0]

#cca_samples = [180.0, 170.0, 100.0, 190.0, 50.0, 150.0, 150.0, 20.0, 0.0, 150.0, 110.0, 160.0, 140.0, 10.0, 120.0, 30.0, -1.0, 0.0, 60.0, 210.0, 600.0, 590.0, 450.0, 270.0, -1.0, -1.0, 690.0, -1.0, 530.0, -1.0, -1.0, -1.0]

In [14]:
samples = np.zeros((64, 2))
print(samples.shape)

for i in range(samples.shape[0]):
    #samples[i, 0] = psda_samples[i]
    samples[i, 0] = m_psda_samples[i]
    samples[i, 1] = cca_samples[i]

(64, 2)


In [15]:
samples_index = 0

sampling_rate = 250
subject = 41
path = "../../TFM-GitHub/Dataset BETA/S" + str(subject) +".mat"
data_frequencies, electrodes_names = utils.load_ssvep_additional_info(path)
occipital_electrodes, occipital_electrodes_indexes, occipital_electrodes_names = utils.select_occipital_electrodes(electrodes_names)

samples_without_stimulus = int(0.5 * sampling_rate)

eeg_data, eeg_data_stimulus_only, trial_length = utils.load_ssvep_data("../../TFM-GitHub/Dataset BETA/",18)

frequencies = [[11, 13], [8.6, 15.6],
               [11, 13], [8.6, 15.6],
               [11, 13], [8.6, 15.6],
               [11, 13], [8.6, 15.6]]


subjects = [18, 18, 64, 64, 11, 11, 67, 67]
cases = ["best", "worst", "best", "worst", "paper_worst_best", "paper_worst_worst", "paper_best_best", "paper_best_worst"]

data_frequencies[-5] = 15.6

electrode_idx = 61
electrode_name = "OZ"

for i in range(len(subjects)):
    subject = subjects[i]
    eeg_data, eeg_data_stimulus_only, trial_length = utils.load_ssvep_data("../../TFM-GitHub/Dataset BETA/",subject)
    case = cases[i]
    for j in range(2):
        index = j+1
        fr_idx = 0
        freq = frequencies[i][j]
        for x in range(40):
            if data_frequencies[x] == freq:
                fr_idx = x
                break
                    
        for tr_idx in range(4):
            path = "../../../Universidad/TFM/Imágenes/Results/"
            path += "S"+str(subject)+"_"+electrode_name+"_"+case+"_"+str(index)+"_T"+str(tr_idx)+".png"
            print(path)
            #utils.plot_generation_three(eeg_data[electrode_idx, :, tr_idx, fr_idx], sampling_rate, fr_idx, data_frequencies, False, True, path, shading_sample = detected_sample(subject, fr_idx, tr_idx), shading_size = window_length)
            utils.plot_generation_three(eeg_data[electrode_idx, :, tr_idx, fr_idx], sampling_rate, fr_idx, data_frequencies, False, True, path, shading_samples = samples[samples_index], shading_size = window_length)

            path = "../../../Universidad/TFM/Imágenes/Results/"
            path += "S"+str(subject)+"_"+electrode_name+"_"+case+"_"+str(index)+"_T"+str(tr_idx)+"_FFT.png"
            utils.plot_generation_fft(eeg_data[electrode_idx, :, tr_idx, fr_idx], sampling_rate=sampling_rate, frequency=data_frequencies[fr_idx], show_plot=False, save_plot=True, path=path)

            samples_index += 1

../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_1_T0.png


/Users/tania/Desktop/Tania/Repositorios GitHub/TFM-GitHub/Sliding windows/utils.py:357: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_1_T1.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_1_T2.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_1_T3.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_2_T0.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_2_T1.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_2_T2.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_best_2_T3.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_1_T0.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_1_T1.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_1_T2.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_1_T3.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_2_T0.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_2_T1.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_2_T2.png
../../../Universidad/TFM/Imágenes/Results/S18_OZ_worst_2_T3.png
../../../Universidad/TFM/Imágen